In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv')

#df['Rating'] = df['Rating'].map({1:0, 2:0, 3:1, 4:1, 5:1})

df.to_csv('data.csv', index=False)  

df.head()

In [ ]:
df.info()

df.describe()

In [ ]:
df.Rating.value_counts()

In [ ]:
import re
import string
from nltk.corpus import stopwords

def remove_URL (text):
    url = re.compile(r"https?://\s+ www\.\s+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)
         

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (i0s)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)


def remove_punct (text):
    table = str.maketrans ("","",string.punctuation)
    return text.translate(table)

stop = set (stopwords.words ( "english"))
def remove_stopwords (text):
    text = [word.lower () for word in text.split() if word.lower() not in stop]
    return " ".join (text)

In [ ]:
def nettoyerDataframe(df):
    Review_cleaned = df.Review.map(lambda x: remove_URL(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_html(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_emoji(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_punct(x)) 
    Review_cleaned = Review_cleaned.map(remove_stopwords)   
    df.insert(1, "Review_cleaned", Review_cleaned)
    return df

In [ ]:
df = nettoyerDataframe(df)

df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))

X = tfidf_vectorizer.fit_transform(df.Review_cleaned)

[ x for x in X.todense()[0][0:].tolist()[0] if x != 0] 

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,precision_score

def afficherMetriques(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    print(f"Accuracy : {accuracy * 100} %")
    print (f"Précision {precision * 100} %") 
    print (f"Recall {recall * 100} %") 

In [ ]:
from sklearn.model_selection import train_test_split

y = df["Rating"].values

X_train , X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [ ]:
#Regression logistique
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression(multi_class='multinomial')
logisticRegression.fit(X_train, y_train)

y_pred = logisticRegression.predict(X_test)

print("\n***** Métriques d'évaluation : Regression logistique ******")
afficherMetriques(y_test,y_pred)

In [ ]:
#Naives Bayes
from sklearn.naive_bayes import GaussianNB

gaussianNB = GaussianNB()
gaussianNB.fit(X_train.toarray(),y_train)

y_pred = logisticRegression.predict(X_test)

print("\n***** Métriques d'évaluation : Naives Bayes ******")
afficherMetriques(y_test,y_pred)

In [ ]:
#SVM
from sklearn import svm

svmModel = svm.SVC(kernel='linear') # Linear Kernel
svmModel.fit(X_train, y_train)

y_pred = svmModel.predict(X_test)

print("\n***** Métriques d'évaluation : SVM ******")
afficherMetriques(y_test,y_pred)

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier()
pac.fit(X_train, y_train)

ypred = pac.predict(X_test)

print("\n***** Métriques d'évaluation : PAC ******")
afficherMetriques(y_test,y_pred)

In [ ]:
#Validation croisée sur le modèle de regression logfistique
from sklearn.model_selection import cross_val_score

rappels = cross_val_score(logisticRegression, X,y, cv=10,scoring='recall')
rappels = pd.Series(rappels)
print("Rappels : min=",rappels.min(),"  max=",rappels.max(),"  moyenne=",rappels.mean())

precisions = cross_val_score(logisticRegression, X,y, cv=10,scoring='precision')
precisions = pd.Series(precisions)
print("Precisions : min=",precisions.min(),"  max=",precisions.max(),"  moyenne=",precisions.mean())

accuracys = cross_val_score(logisticRegression, X,y, cv=10,scoring='accuracy')
accuracys = pd.Series(accuracys)
print("Accuracy : min=",accuracys.min(),"  max=",accuracys.max(),"  moyenne=",accuracys.mean())